In [2]:
!pip install mmcv-full
!git clone https://github.com/open-mmlab/mmdetection.git
!cd mmdetection; python setup.py install

  Using cached mmcv-full-1.3.13.tar.gz (313 kB)
     |████████████████████████████████| 185 kB 5.2 MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.13-cp37-cp37m-linux_x86_64.whl size=21808725 sha256=c21691e55d64653ef3be3f7a01afa6b3bce0f718d59ace038a68235f7439829a
  Stored in directory: /root/.cache/pip/wheels/9d/13/23/7c772e752c5e05ef7fc7328dccefc152df8ab7bbf66b2ee59c
Successfully built mmcv-full
Cloning into 'mmdetection'...
remote: Enumerating objects: 20899, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 20899 (delta 1), reused 9 (delta 1), pack-reused 20886
Receiving objects: 100% (20899/20899), 24.66 MiB | 25.90 MiB/s, done.
Resolving deltas: 100% (14628/14628), done.
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
running install
running bdist_egg
running egg_info
creating mmdet.egg-info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing

In [3]:
from mmdet.apis import init_detector,inference_detector
import mmcv

### **Oxford Pet Dataset 다운로드**
image와 annotations을 압축파일로 각각 doenload수행

In [4]:
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

--2021-09-16 05:48:24--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/x-gzip]
Saving to: ‘images.tar.gz’

images.tar.gz       100%[===================>] 755.23M  29.1MB/s    in 27s     

2021-09-16 05:48:52 (27.9 MB/s) - ‘images.tar.gz’ saved [791918971/791918971]

--2021-09-16 05:48:52--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19173078 (18M) [application/x-gzip]
Saving to: ‘annotations.tar.gz’

annotations.tar.gz  100%[===================>]  18.28M  14.1MB/s    in 1.3s    

2021-09-16 05:48:53 (14.1 M

In [5]:
# /content/data 디렉토리를 만들고 해당 디렉토리에 다운로드 받은 압축 파일 풀기.
!mkdir /content/data
!tar -xvf images.tar.gz -C /content/data
!tar -xvf annotations.tar.gz -C /content/data

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
annotations/trimaps/._pomeranian_180.png
annotations/trimaps/pomeranian_180.png
annotations/trimaps/._pomeranian_181.png
annotations/trimaps/pomeranian_181.png
annotations/trimaps/._pomeranian_182.png
annotations/trimaps/pomeranian_182.png
annotations/trimaps/._pomeranian_183.png
annotations/trimaps/pomeranian_183.png
annotations/trimaps/._pomeranian_184.png
annotations/trimaps/pomeranian_184.png
annotations/trimaps/._pomeranian_185.png
annotations/trimaps/pomeranian_185.png
annotations/trimaps/._pomeranian_186.png
annotations/trimaps/pomeranian_186.png
annotations/trimaps/._pomeranian_187.png
annotations/trimaps/pomeranian_187.png
annotations/trimaps/._pomeranian_188.png
annotations/trimaps/pomeranian_188.png
annotations/trimaps/._pomeranian_189.png
annotations/trimaps/pomeranian_189.png
annotations/trimaps/._pomeranian_19.png
annotations/trimaps/pomeranian_19.png
annotations/trimaps/._pomeranian_190.png
annotations/trimaps/pomeranian_190.png
annota

### **이미지 디렉토리와 annotation파일 살펴 보기**

In [6]:
!ls -lia ./data/images/Abyss*.jpg

2495485 -rwxr-xr-x 1 1001 1001 126923 Jun 18  2012 ./data/images/Abyssinian_100.jpg
2498030 -rwxr-xr-x 1 1001 1001  49023 Jun 18  2012 ./data/images/Abyssinian_101.jpg
2499106 -rwxr-xr-x 1 1001 1001 165505 Jun 18  2012 ./data/images/Abyssinian_102.jpg
2499878 -rwxr-xr-x 1 1001 1001 132937 Jun 18  2012 ./data/images/Abyssinian_103.jpg
2500028 -rwxr-xr-x 1 1001 1001  34907 Jun 18  2012 ./data/images/Abyssinian_104.jpg
2498409 -rwxr-xr-x 1 1001 1001 141823 Jun 18  2012 ./data/images/Abyssinian_105.jpg
2494451 -rwxr-xr-x 1 1001 1001 197092 Jun 18  2012 ./data/images/Abyssinian_106.jpg
2499919 -rwxr-xr-x 1 1001 1001 150955 Jun 18  2012 ./data/images/Abyssinian_107.jpg
2496860 -rwxr-xr-x 1 1001 1001 120641 Jun 18  2012 ./data/images/Abyssinian_108.jpg
2498398 -rwxr-xr-x 1 1001 1001  13601 Jun 18  2012 ./data/images/Abyssinian_109.jpg
2493820 -rwxr-xr-x 1 1001 1001  96294 Jun 18  2012 ./data/images/Abyssinian_10.jpg
2496813 -rwxr-xr-x 1 1001 1001  27734 Jun 18  2012 ./data/images/Abyssinian_1

In [7]:
!ls -lia ./data/images

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2495312 -rwxr-xr-x 1 1001 1001   27987 Jun 18  2012 Egyptian_Mau_91.jpg
2501021 -rwxr-xr-x 1 1001 1001   31662 Jun 18  2012 Egyptian_Mau_92.jpg
2500043 -rwxr-xr-x 1 1001 1001   11833 Jun 18  2012 Egyptian_Mau_93.jpg
2496290 -rwxr-xr-x 1 1001 1001   13282 Jun 18  2012 Egyptian_Mau_94.jpg
2499030 -rwxr-xr-x 1 1001 1001   25425 Jun 18  2012 Egyptian_Mau_95.jpg
2494269 -rwxr-xr-x 1 1001 1001   22833 Jun 18  2012 Egyptian_Mau_96.jpg
2494806 -rwxr-xr-x 1 1001 1001  181499 Jun 18  2012 Egyptian_Mau_97.jpg
2497081 -rwxr-xr-x 1 1001 1001    8709 Jun 18  2012 Egyptian_Mau_98.jpg
2497287 -rwxr-xr-x 1 1001 1001   12394 Jun 18  2012 Egyptian_Mau_99.jpg
2498742 -rwxr-xr-x 1 1001 1001   48984 Jun 18  2012 Egyptian_Mau_9.jpg
2498883 -rwxr-xr-x 1 1001 1001  164254 Jun 18  2012 english_cocker_spaniel_100.jpg
2497196 -rwxr-xr-x 1 1001 1001  170083 Jun 18  2012 english_cocker_spaniel_101.jpg
2497791 -rwxr-xr-x 1 1001 1001  241519 Jun 18  2012 english_cocker_spaniel_102.

In [9]:
!cat ./data/annotations/xmls/Abyssinian_1.xml

<annotation><folder>OXIIIT</folder><filename>Abyssinian_1.jpg</filename><source><database>OXFORD-IIIT Pet Dataset</database><annotation>OXIIIT</annotation><image>flickr</image></source><size><width>600</width><height>400</height><depth>3</depth></size><segmented>0</segmented><object><name>cat</name><pose>Frontal</pose><truncated>0</truncated><occluded>0</occluded><bndbox><xmin>333</xmin><ymin>72</ymin><xmax>425</xmax><ymax>158</ymax></bndbox><difficult>0</difficult></object></annotation>


In [15]:
import glob
import xml.etree.ElementTree as ET

def get_bboxes_from_xml_test(xml_file):
  tree=ET.parse(xml_file)
  root=tree.getroot()
  bbox_names=[]
  bboxes=[]

  for obj in root.findall('object'):
    bbox_name=obj.find('name').text
    xmlbox=obj.find('bndbox')
    x1=int(xmlbox.find('xmin').text)
    y1=int(xmlbox.find('ymin').text)
    x2=int(xmlbox.find('xmax').text)
    y2=int(xmlbox.find('ymax').text)

    bboxes.append([x1,y1,x2,y2])
    bbox_names.append(bbox_name)

    return bbox_names,bboxes

get_bboxes_from_xml_test('./data/annotations/xmls/Abyssinian_1.xml')

(['cat'], [[333, 72, 425, 158]])

### **train,val image/annotation 메타 파일 보기**

In [17]:
import pandas as pd
pet_df=pd.read_csv('/content/data/annotations/trainval.txt',
                   sep=' ',
                   header=None,
                   names=['img_name','class_id','etc1','etc2'])
pet_df.head()

,img_name,class_id,etc1,etc2
0,Abyssinian_100,1,1,1
1,Abyssinian_101,1,1,1
2,Abyssinian_102,1,1,1
3,Abyssinian_103,1,1,1
4,Abyssinian_104,1,1,1


In [18]:
pet_df['class_id'].value_counts()

37    100
22    100
34    100
32    100
30    100
28    100
26    100
24    100
20    100
35    100
18    100
16    100
14    100
10    100
6     100
4     100
36    100
1     100
3     100
19    100
31    100
29    100
27    100
25    100
5     100
21    100
17    100
15    100
11    100
9     100
7     100
2     100
33     99
23     96
13     96
8      96
12     93
Name: class_id, dtype: int64

In [19]:
pet_df['class_name']=pet_df['img_name'].apply(lambda x: x[:x.rfind('_')])
pet_df.head()

,img_name,class_id,etc1,etc2,class_name
0,Abyssinian_100,1,1,1,Abyssinian
1,Abyssinian_101,1,1,1,Abyssinian
2,Abyssinian_102,1,1,1,Abyssinian
3,Abyssinian_103,1,1,1,Abyssinian
4,Abyssinian_104,1,1,1,Abyssinian


In [22]:
from sklearn.model_selection import train_test_split
train_df,val_df=train_test_split(pet_df,test_size=0.1,stratify=pet_df['class_id'],random_state=2021)

In [23]:
print(train_df['class_id'].value_counts(), val_df['class_id'].value_counts())


37    90
22    90
34    90
32    90
30    90
28    90
26    90
24    90
20    90
35    90
18    90
16    90
14    90
10    90
6     90
4     90
36    90
1     90
3     90
19    90
31    90
29    90
27    90
25    90
5     90
21    90
17    90
15    90
11    90
9     90
7     90
2     90
33    89
23    87
13    86
8     86
12    84
Name: class_id, dtype: int64 37    10
36    10
17    10
16    10
15    10
14    10
13    10
11    10
10    10
9     10
8     10
7     10
6     10
5     10
4     10
3     10
2     10
18    10
19    10
20    10
21    10
35    10
34    10
33    10
32    10
31    10
30    10
29    10
28    10
27    10
26    10
25    10
24    10
22    10
1     10
12     9
23     9
Name: class_id, dtype: int64


In [24]:
train_df = train_df.sort_values(by='img_name')
val_df = val_df.sort_values(by='img_name')

In [25]:
# ann_file로 주어지는 메타파일은 가급적이면 소스데이터의 가장 상단 디렉토리에 저장하는 것이 바람직. 
train_df['img_name'].to_csv('./data/train.txt', sep=' ', header=False, index=False)
val_df['img_name'].to_csv('./data/val.txt', sep=' ', header=False, index=False)

In [26]:
pet_classes_list=pet_df['class_name'].unique().tolist()
print(pet_classes_list)

['Abyssinian', 'american_bulldog', 'american_pit_bull_terrier', 'basset_hound', 'beagle', 'Bengal', 'Birman', 'Bombay', 'boxer', 'British_Shorthair', 'chihuahua', 'Egyptian_Mau', 'english_cocker_spaniel', 'english_setter', 'german_shorthaired', 'great_pyrenees', 'havanese', 'japanese_chin', 'keeshond', 'leonberger', 'Maine_Coon', 'miniature_pinscher', 'newfoundland', 'Persian', 'pomeranian', 'pug', 'Ragdoll', 'Russian_Blue', 'saint_bernard', 'samoyed', 'scottish_terrier', 'shiba_inu', 'Siamese', 'Sphynx', 'staffordshire_bull_terrier', 'wheaten_terrier', 'yorkshire_terrier']


In [27]:
!echo 'train list #####'; cat ./data/train.txt

train list #####
Abyssinian_1
Abyssinian_10
Abyssinian_101
Abyssinian_102
Abyssinian_103
Abyssinian_104
Abyssinian_105
Abyssinian_106
Abyssinian_107
Abyssinian_108
Abyssinian_109
Abyssinian_110
Abyssinian_111
Abyssinian_112
Abyssinian_113
Abyssinian_114
Abyssinian_115
Abyssinian_116
Abyssinian_117
Abyssinian_118
Abyssinian_119
Abyssinian_12
Abyssinian_120
Abyssinian_121
Abyssinian_123
Abyssinian_124
Abyssinian_125
Abyssinian_126
Abyssinian_128
Abyssinian_129
Abyssinian_13
Abyssinian_130
Abyssinian_131
Abyssinian_132
Abyssinian_133
Abyssinian_134
Abyssinian_135
Abyssinian_136
Abyssinian_137
Abyssinian_138
Abyssinian_139
Abyssinian_14
Abyssinian_141
Abyssinian_142
Abyssinian_143
Abyssinian_144
Abyssinian_145
Abyssinian_146
Abyssinian_148
Abyssinian_149
Abyssinian_15
Abyssinian_150
Abyssinian_151
Abyssinian_152
Abyssinian_154
Abyssinian_155
Abyssinian_156
Abyssinian_157
Abyssinian_159
Abyssinian_16
Abyssinian_160
Abyssinian_161
Abyssinian_164
Abyssinian_165
Abyssinian_166
Abyssinian_167
A

In [28]:
!echo ' valid list ###'; cat ./data/val.txt

 valid list ###
Abyssinian_100
Abyssinian_11
Abyssinian_122
Abyssinian_127
Abyssinian_140
Abyssinian_153
Abyssinian_158
Abyssinian_182
Abyssinian_190
Abyssinian_193
Bengal_112
Bengal_127
Bengal_128
Bengal_139
Bengal_143
Bengal_145
Bengal_149
Bengal_169
Bengal_180
Bengal_190
Birman_113
Birman_120
Birman_123
Birman_145
Birman_149
Birman_151
Birman_159
Birman_160
Birman_171
Birman_190
Bombay_10
Bombay_110
Bombay_121
Bombay_122
Bombay_129
Bombay_133
Bombay_140
Bombay_143
Bombay_168
Bombay_174
British_Shorthair_103
British_Shorthair_136
British_Shorthair_141
British_Shorthair_15
British_Shorthair_155
British_Shorthair_17
British_Shorthair_18
British_Shorthair_185
British_Shorthair_199
British_Shorthair_209
Egyptian_Mau_100
Egyptian_Mau_102
Egyptian_Mau_111
Egyptian_Mau_118
Egyptian_Mau_127
Egyptian_Mau_154
Egyptian_Mau_161
Egyptian_Mau_192
Egyptian_Mau_193
Maine_Coon_112
Maine_Coon_129
Maine_Coon_132
Maine_Coon_153
Maine_Coon_173
Maine_Coon_193
Maine_Coon_196
Maine_Coon_197
Maine_Coon_207
M